# Benchmarks

## Initialize

In [ ]:
%load_ext autoreload
%autoreload 2

import os
from tqdm.auto import tqdm
import pathlib

import numpy as np
import pandas as pd
import lifelines

In [ ]:
%env MKL_NUM_THREADS=1
%env NUMEXPR_NUM_THREADS=1
%env OMP_NUM_THREADS=1

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=40, threads_per_worker=1)
client = Client(cluster)
cluster.scheduler

In [ ]:
project_name = "210616_centres_dask"
data_path = "/data/analysis/ag-reils/steinfej"
data_pre = f"{data_path}/data/2_datasets_pre/{project_name}"
data_post = f"{data_path}/data/3_datasets_post/{project_name}"

project_label = "21_PGS_Revision"
project_path = f"/data/analysis/ag-reils/ag-reils-shared/cardioRS/results/projects/{project_label}"
figures_path = f"{project_path}/figures"
data_results_path = f"{project_path}/data"
pathlib.Path(figures_path).mkdir(parents=True, exist_ok=True)
pathlib.Path(data_results_path).mkdir(parents=True, exist_ok=True)

In [ ]:
data =  pd.read_feather(f"{data_post}/data_merged.feather")

In [ ]:
endpoints = ['MACE']
endpoint_labels = sorted([f"{e}_event" for e in endpoints]+[f"{e}_event_time" for e in endpoints])
endpoint_data =  pd.read_feather(f"{data_post}/data_merged.feather", columns=["eid"] + endpoint_labels)

In [ ]:
preds = pd.read_feather(f"{data_results_path}/predictions_210703_centres_FINAL.feather")

In [ ]:
data_test = preds[['eid','endpoint', 'module','features','split','partition','Ft_10']].query("split=='test'")
data_test

modules = data_test.module.unique().tolist()
features = data_test.features.unique().tolist()
partitions = data_test.partition.unique().tolist()

In [ ]:
iterations=[i for i in range(1000)]

In [ ]:
from IPython.display import clear_output
from sksurv.metrics import concordance_index_ipcw, brier_score, cumulative_dynamic_auc, integrated_brier_score
from lifelines.utils import concordance_index
from dask.diagnostics import ProgressBar

def calculate_per_endpoint(df, endpoint, module, feature, iteration, time):  
    event = [0 if (endpoint_event == 0) | (endpoint_event_time > time) else 1 for endpoint_event, endpoint_event_time in zip(df[endpoint+"_event"], df[endpoint+"_event_time"])]
    event_time = [time if (endpoint_event == 0) | (endpoint_event_time > time) else endpoint_event_time for endpoint_event, endpoint_event_time in zip(df[endpoint+"_event"], df[endpoint+"_event_time"])]
    df = df.assign(event = event, event_time = event_time)
    df = df.dropna(subset=["event_time", f"Ft_{time}", "event"], axis=0)
    
    cindex = 1-concordance_index(df["event_time"], df[f"Ft_{time}"], df["event"])
    #except: cindex=np.nan
    return {"endpoint":endpoint, "module": module, "features": feature, "iteration": iteration, "time":time, "cindex":cindex}

data_bm = data_test.query("endpoint==@endpoint").merge(endpoint_data[["eid", f"{endpoint}_event", f"{endpoint}_event_time"]], on="eid", how="left")
eids = data_test.eid.unique()
endpoint = "MACE"
with ProgressBar():
    rows = []
    for iteration in tqdm(iterations): 
        eids_bs = np.random.choice(eids, size=len(eids))
        for module in modules: 
            temp_module = data_bm.query("module==@module")
            for feature in features:
                temp_features = temp_module.query("features==@feature")
                if len(temp_features)>0:
                    data_future = client.scatter(temp_features[["eid", "Ft_10", "MACE_event", "MACE_event_time"]].set_index("eid").loc[eids_bs].reset_index())
                    rows.append(client.submit(calculate_per_endpoint, data_future, endpoint, module, feature, iteration, time=10))

In [ ]:
from dask.distributed import progress
progress(rows)

In [ ]:
rows = client.gather(rows)
benchmark_endpoints_pp = pd.DataFrame({"endpoint":[], "module": [], "features": [], "partition": [], "time": [], "cindex": []}).append(rows, ignore_index=True)
clear_output()

In [ ]:
name = "benchmark_cindex_MACE_210703_all_FINAL"
benchmark_endpoints_pp.to_feather(f"{data_results_path}/{name}.feather")